# JEM092 Asset Pricing - Homework 2
### Summer Semester 2021/2022

### Authors: *Petr Čala, Tereza Čechová, Vilém Krejcar*
___

First things first, we install and load the necessary packages. Then we arbitrarily set a seed for reproducibility.

In [220]:
# Required packages
packages <- c("stringr", "quantmod", "tseries", "dplyr", "xml2", "httr", "PortfolioAnalytics",
              "lubridate", "sandwich", "hash")

# Install packages not yet installed
installed_packages <- packages %in% rownames(installed.packages())
if (any(installed_packages == FALSE)) {
  install.packages(packages[!installed_packages])
  print(paste("Installing package ", packages[!installed_packages],"...", sep = ""))
}

# Packages loading
invisible(lapply(packages, library, character.only = TRUE))
rm(list = ls()) #Clean environment
print('All packages loaded successfully...')

# Miscellaneous
options(repr.plot.width = 6, repr.plot.height = 5)
set.seed(420)

[1] "All packages loaded successfully..."


### Groundwork

Although we were able to obtain the data in the first homework, **we choose to use the provided dataset**, as this allows for easier and **more reliable reproducibility**. First we load the data, subset the necessary stocks, save these as new variables, and then discard the source data in order to alleviate some memory.

In [221]:
# Static variables
group_number <- 61505008
ticker_file <- "data/61505008_data_download.csv"
data_file <- "data/Asset_Pricing_HW_2_data.RData"
group_factor <- "data/third_factor_rand.csv"
data_files <- list(ticker_file, data_file, group_factor)

In [222]:
 # Set wd to project root
if (str_detect(getwd(), 'scripts')) {
    setwd('..')
}

# Assert presence of all required files
for (file in data_files) {
    if (!file.exists(file)) {
        print(paste0('File ', file, ' does not exist or has been misplaced.'))
    }
}

# Load tickers of required stocks
tickers <- as.character(unlist(read.csv(ticker_file)[2]))

# Check the assigned group factor
getFactor <- function(factor){
    data <- read.csv(factor)
    matching_row <- which(data[2] == group_number)
    assigned_factor <- as.character(data[matching_row, 3])
    print(paste0('The factor assigned to this group is ', assigned_factor, '.'))
}
getFactor(group_factor)

[1] "The factor assigned to this group is VOL."


While selecting only the desired tickers, we found that **a ticker that was assigned to our group was missing from the provided data**. We decided to remedy for this by writing a simple fix which replaces all of these missing assigned data by random data of a stock, that is provided in the source data set. As this was the case for only one stock, no trend bias should be introduced into the data.

In [223]:
# Load source data into the working directory
source_data <- load(data_file)

# Subset only for the stocks we will need
# Note - use these as BV$AAPL, not BV['AAPL']
BV <- book_value_sap500[tickers]
MKT <- MktCap_sap500[tickers]
OHLCV <- OHLCV_sap500[tickers]

# Assert presence of all assigned tickers in the source data - replace data of those that are missing
data_replacement_counter <- 1
for (ticker in tickers) {
    all_stock_names <- names(OHLCV)
    if (!ticker %in% all_stock_names) { # Missing file found
        print(paste0(ticker, ' is missing from the source data...'))
        missing_ticker_idx <- which(tickers == ticker) # Index of the missing file
        
        # Get replacement data
        while (names(book_value_sap500[data_replacement_counter]) %in% tickers) {
            data_replacement_counter <- data_replacement_counter + 1 # Search for a stock not from the assigned set
        }

        # Replace the data
        replacement_stock <- names(book_value_sap500)[data_replacement_counter] # Stock which shall serve as a replacement
        all_stock_names[missing_ticker_idx] <- replacement_stock # Fix names

        BV[missing_ticker_idx] <- book_value_sap500[replacement_stock]
        MKT[missing_ticker_idx] <- MktCap_sap500[replacement_stock]
        OHLCV[missing_ticker_idx] <- OHLCV_sap500[replacement_stock]

        names(BV) <- all_stock_names
        names(MKT) <- all_stock_names
        names(OHLCV) <- all_stock_names
        print(paste0('Data for ', ticker, ' replaced successfully with data from ', replacement_stock))

        data_replacement_counter <- data_replacement_counter + 1
    }
}

# Remove source data from working directory for better memory management
rm(list = source_data)

[1] "MOH is missing from the source data..."
[1] "Data for MOH replaced successfully with data from ABT"


### Data preprocessing

We define several functions to help us preprocess data.

In [224]:
######################################
### Data loading and preprocessing ###
######################################

# Download the desired factor and return it as a single xts object
#   factor_name(str) - Name of the factor to download. Accepts the following values:
#       -> 'FF3_daily', 'FF3', 'Mom', 'Illiq'
#   latest_expected_date(str) - Date which the dartmouth website currently provides as the latest
#
#   Note: If 'daily' is explicitly not defined in the factor_name, the downloaded factor will contain
#       monthly values
getFactor <- function(factor_name, latest_expected_date = '2022-03-31') {
    # Static variables #
    latest_expected_date <- as.Date(as.yearmon(latest_expected_date), frac = 1)

    # Path to the file on the website
    web_file_paths <- hash() 
    web_file_paths[['FF3_daily']] <- "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_TXT.zip"
    web_file_paths[['FF3']] <- "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_TXT.zip"
    web_file_paths[['Mom']] <- "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Momentum_Factor_TXT.zip"
    web_file_paths[['Illiq']] <- "https://faculty.chicagobooth.edu/-/media/faculty/lubos-pastor/data/liq_data_1962_2021.txt"

    # Check input validity
    if (!factor_name %in% keys(web_file_paths)) {
        stop('Invalid input specification. The function is not able to handle this input.')
    }

    # Path to where the file should be downloaded to and stored in
    folder_file_names <- hash()
    folder_file_names[['FF3_daily']] <- 'data/F-F_Research_Data_Factors_daily.zip'
    folder_file_names[['FF3']] <- 'data/F-F_Research_Data_Factors.zip'
    folder_file_names[['Mom']] <- 'data/F-F_Momentum_Factor.zip'
    folder_file_names[['Illiq']] <- 'data/Illiquidity_Factor.txt'

    # Column names to be extracted for each factor
    desired_cols <- hash()
    desired_cols[['FF3_daily']] <- c('t', 'Mkt-RF', 'SMB', 'HML', 'RF') 
    desired_cols[['FF3']] <- c('t', 'Mkt-RF', 'SMB', 'HML', 'RF') 
    desired_cols[['Mom']] <- c('t', 'Mom')
    desired_cols[['Illiq']] <- c('t', 'Agg Liq', 'Innov Liq', 'Traded Liq')

    # N of rows of each file to read
    desired_rows <- hash()
    desired_rows[['FF3_daily']] <- 25210
    desired_rows[['FF3']] <- 1149
    desired_rows[['Mom']] <- 1143
    desired_rows[['Illiq']] <- 713

    # Number of rows to be skipped
    skip_rows <- hash()
    skip_rows[['FF3_daily']] <- 5
    skip_rows[['FF3']] <- 4
    skip_rows[['Mom']] <- 14
    skip_rows[['Illiq']] <- 11

    # Data download, saving and loading
    web_file_path <- web_file_paths[[factor_name]]
    folder_file_path <- folder_file_names[[factor_name]]
    if (!file.exists(folder_file_path)){ # Download if doesn't exist
        download.file(web_file_path, destfile = folder_file_path)
    }
    if (grepl('.zip', folder_file_path, fixed=TRUE)){ # Unzip if file is .zip file
        unzip(folder_file_path, exdir = "data")
    }

    txt_path <- gsub('.zip', '.txt', folder_file_path) # Get .txt file path (data file)
    cols <- desired_cols[[factor_name]]
    rows <- desired_rows[[factor_name]]
    skip <- skip_rows[[factor_name]]

    factors_raw <- read.delim( # Read data into a var
        txt_path,
        col.names = cols,
        sep = '',
        nrows = rows,
        header = FALSE,
        skip = skip,
        stringsAsFactors = FALSE
    )
    
    date_chars <- as.character(factors_raw[['t']])
    if (!grepl('daily', factor_name, fixed = TRUE)){ # Adding a placeholder date for monthly data
        date_chars <- paste0(date_chars, '01')
    }
    factors_raw[['t']] <- as.Date(date_chars, '%Y%m%d') # Time index to date
    factors <- as.xts(factors_raw[, -1], order.by = factors_raw[['t']]) # To xts
    factors <- factors / 100
    if (!grepl('daily', factor_name, fixed = TRUE)){ # Get last day of month, if monthly index
        index(factors) <- as.Date(as.yearmon(index(factors)), frac = 1)
    }
    # Check if row subsetting is correct - Illiquidity provides different data, skip the check there
    latest_date_in_data <- tail(index(factors), n = 1)
    if(!latest_expected_date == latest_date_in_data & !factor_name == 'Illiq') {
        print('Latest date specified does not match the data last value. Check number of rows in the file.')
        print(paste0('The latest available index is ', latest_date_in_data))
    }
    return(factors)
}

# Input an xts object with daily data of market capitalization and return this as an xts data frame
#   with monthly market capitalization.
getMonthlyMarketCap <- function(daily_market_cap_data) {
    stock_names <- names(daily_market_cap_data)
    daily_market_caps <- do.call('cbind', daily_market_cap_data)
    monthly_market_caps <- apply.monthly(daily_market_caps, tail, 1)
    colnames(monthly_market_caps) <- stock_names
    index(monthly_market_caps) <- as.Date(as.yearmon(index(monthly_market_caps)), frac = 1)
    return(monthly_market_caps)
}

# Input a list of OHLCV datasets and return a similar object, only subsetted for adjusted prices. Discard the rest of the columns
getAdjustedPrices <- function(OHLCV_data) {
    if (!class(OHLCV_data) == 'list') {
        print('The input data must be a list')
        break
    }
    yankPrices <- function(data) {
        adj_col_idx <- which(str_detect(colnames(data), 'Adjusted')) # Get index of column containing adjusted prices
        prices <- data[, adj_col_idx]
        return(prices)
    }
    data_out <- lapply(OHLCV_data, yankPrices)
    data_out <- do.call('cbind', data_out)
    return(data_out)
}

# Input an xts data frame containing raw adjusted prices and convert these to daily or monthly returns
getReturns <- function(adjusted_prices_data, type = 'monthly') {
    if (!type %in% c('daily', 'monthly')) {
        stop('Can only handle daily or monthly returns calculation...')
    }
    getReturn <- function(data) {
        if (type == 'monthly') {
            return(suppressWarnings(monthlyReturn(data)))
        } else {
            return(suppressWarnings(dailyReturn(data)))
        }
    }    
    returns <- lapply(adjusted_prices_data, getReturn) # Calculate returns for all stocks
    returns <- do.call('cbind', returns)
    colnames(returns) <- gsub('Adjusted', 'returns', names(adjusted_prices_data)) # Rename columns
    return(returns)
}

#########################################
### Performance indicator calculation ###
#########################################

# Input daily and monthly returns data, along with daily ff3 factors, all as an xts object.
#   Return an xts object containing monthly betas for the input, calculated using last 12 months of data.
getMonthlyBetas <- function(daily_returns_data, monthly_returns_data, daily_ff3_data) {
    names(monthly_returns_data) <- gsub('returns', 'betas', names(monthly_returns_data))
    stocks <- names(monthly_returns_data) # Stock names - AAPL.betas
    monthly_dates <- index(monthly_returns_data) # Dates by months

    monthly_betas <- lapply(monthly_dates, function(end_date) { #Betas for all
        start_date <- end_date %m-% months(12) # Last 12 months of data
        date_range <- paste0(start_date, '/', end_date)
        daily_returns <- daily_returns_data[date_range]
        market_returns <- daily_ff3_data$mkt_rf[date_range]
        stocks_betas <- lapply(daily_returns, function(stock_returns) {
            model_data <- na.omit(cbind(stock_returns, market_returns))
            if (nrow(model_data) < 12 * 15) {
                return(NA)
            }
            model_betas <- lm(model_data[,1] ~ model_data[,2])
            return(model_betas$coefficients[[2]]) # Beta for given stock
        })
        return(unlist(stocks_betas)) # Betas for all stocks, given time frame
    })
    monthly_betas <- do.call('rbind', monthly_betas)
    monthly_betas <- as.xts(monthly_betas, order.by = monthly_dates)
    colnames(monthly_betas) <- stocks
    return(monthly_betas)
}

# Input a list of xts objects containing market capitalization info and convert these to monthly sizes
getMonthlySizes <- function(mkt_cap_data) {
    monthly_sizes <- lapply(mkt_cap_data, log) # Calculate monthly sizes
    monthly_sizes <- do.call('cbind', monthly_sizes)
    monthly_sizes <- as.xts(apply(monthly_sizes, 2, function(x) ifelse(is.finite(x), x, NA))) # inf to NA
    colnames(monthly_sizes) <- paste0(colnames(monthly_sizes), '.sizes') # Rename columns
    return(monthly_sizes)
}

# Input two data frames (containing daily and monthly returns data) and return a data frame with monthly volatility
getMonthlyVolatilities <- function(daily_returns_data, monthly_returns_data) {
    names(monthly_returns_data) <- gsub('returns', 'volatility', names(monthly_returns_data))
    stocks <- names(monthly_returns_data) # Stock names - AAPL.returns
    monthly_dates <- index(monthly_returns_data) # Dates by months

    monthly_volatilities <- lapply(monthly_dates, function(end_date) { # Volatilities for all time frames, all stocks
        start_date <- end_date %m-% months(12) # Use last 12 months of data as suggested in the setup
        date_range <- paste0(start_date, '/', end_date)
        daily_returns <- daily_returns_data[date_range]
        stocks_volatilities <- lapply(daily_returns, function(stock) {
            average_stock_return <- mean(stock)
            nominator <- sum((stock - average_stock_return) ^ 2)
            denominator <- length(stock) - 1
            monthly_volatility <- 100 * sqrt(nominator/denominator) * sqrt(12)
            return(monthly_volatility) # Volatility for a given time frame, one stock
        })
        return(unlist(stocks_volatilities)) # Volatility for a given time frame, all stocks
    })
    monthly_volatilities <- do.call('rbind', monthly_volatilities) # Data frame of volatilities
    monthly_volatilities <- as.xts(monthly_volatilities, order.by = monthly_dates) # To xts
    colnames(monthly_volatilities) <- stocks
    return(monthly_volatilities)
}

In [225]:
# Compute the various metrics using previously defined functions
adjusted_prices <- getAdjustedPrices(OHLCV)
daily_returns <- getReturns(adjusted_prices, type = 'daily')
monthly_returns <- getReturns(adjusted_prices, type = 'monthly')
monthly_market_caps <- getMonthlyMarketCap(MKT)
# Download factors from online webistes and transform them into a pretty xts
daily_ff3 <- getFactor('FF3_daily')
monthly_ff3 <- getFactor('FF3')
monthly_mom <- getFactor('Mom')
monthly_illiq <- getFactor('Illiq')

In [8]:
# Compute the three desired factors (monthly)
monthly_sizes <- getMonthlySizes(monthly_market_caps) # 5s
monthly_betas <- getMonthlyBetas(daily_returns, monthly_returns, daily_ff3) # 2 mins
monthly_volatilities <- getMonthlyVolatilities(daily_returns, monthly_returns) # Monthly volatilities (vectors go brrrrrrr) - 9s

### Univariate portfolio analysis

In [9]:
# Input monthly data to sort by, factor name, monthly returns data, and monthly market capital data.
#   Furthermore specify the number of quantiles and the method by which to compute the returns.
#   For this, two options are available - 'equal' (as in equally weighted) and 'value' (as in
#       value weighted).
#   Return overall average returns within portfolios and the corresponding standard errors
#       as a simple data frame.
#   If 'get_quintile_values' == TRUE, return quintile values for this factor instead.
getUnivariateSort <- function(sort_variable_data, factor_name, monthly_returns_data,
                                monthly_market_caps, monthly_ff3_data, n = 5, method = 'equal',
                                get_quintile_values = FALSE){
    # Static
    diff_portfolio <- paste0(n, ' - 1')
    portfolio_colnames <- c(1:n, diff_portfolio)
    portfolio_returns_index <- index(sort_variable_data)[-1] # All but first element

    # Iterate over the months in the portfolio returns index and return return for each of them
    portfolio_returns <- lapply(portfolio_returns_index, function(current_month) {
        this_month_sort_variable <- sort_variable_data[current_month]
        not_na <- !is.na(this_month_sort_variable)
        next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)
        
        # Check for data validity, return a vector of NAs in case of invalid data
        skip_conditions <- c(
            sum(not_na) < 4*n, # Not enough observations for this month
            !next_month %in% index(monthly_returns_data) # No returns data in the next month
            
        )
        if (method == 'value') { # Skip conditions that apply only for the value weighted method
            # No market capitalization data for the next month - nothing to weigh by
            skip_conditions <- append(skip_conditions, !next_month %in% index(monthly_market_caps)) 
        }
        if (any(skip_conditions)) {
            vec_length <- ifelse (get_quintile_values, n, n+1) # No diff in getting quintile values
            return(c(rep(NA, vec_length))) # Return vector of NAs
        }

        next_month_returns <- monthly_returns_data[next_month] # A vector for all stocks
        next_month_market_caps <- monthly_market_caps[next_month]
        next_month_market_caps[is.na(next_month_market_caps)] <- 0 # No weights for missing obs

        breakpoints <- quantile(this_month_sort_variable, prob = 0:n/n, na.rm = TRUE) # Quintiles

        # Compute portfolio returns
        this_period_returns <- c()
        this_period_quintile_averages <- c() # For storing quintile values
        for (i in breakpoints) { # Find all quintile breakpoints
            next_breakpoint_idx <- match(i, breakpoints) + 1
            next_breakpoint <- breakpoints[next_breakpoint_idx]
            if (!is.na(next_breakpoint)) { # While not out of quntiles
                filter <- (i < this_month_sort_variable) &
                          (this_month_sort_variable < next_breakpoint) &
                          (not_na)
                if (method == 'equal') {# Equally weighted portfolio
                    quintile_return <- mean(next_month_returns[,filter])
                } else if (method == 'value') {
                    quintile_return <- weighted.mean(t(next_month_returns[,filter]),
                                                    t(next_month_market_caps[,filter])) # Actual return
                } else {
                    stop('Incorrectly defined method. Select one of - equal, value.')
                }
                if (get_quintile_values) { # Store average factor value in this quintile
                    this_period_quintile_averages <- append(this_period_quintile_averages,
                        mean(this_month_sort_variable[,filter]))
                }
                this_period_returns <- append(this_period_returns, quintile_return) # Store value 
            } else {
                break # Last iteration
            } 
        }
        if (get_quintile_values) {
            return(this_period_quintile_averages) # Return quintile averages instead
        }
        diff_returns <- this_period_returns[[n]] - this_period_returns[[1]]
        this_period_returns <- append(this_period_returns, diff_returns) # Append returns of diff
        return(this_period_returns) # Returns for all quintiles for this period
    })
    portfolio_returns <- as.xts(do.call('rbind', portfolio_returns), # To xts
        order.by = as.Date(as.yearmon(portfolio_returns_index), frac = 1))
    
    if (get_quintile_values){ # Calculate quintile values instead
        quintile_values <- lapply(portfolio_returns, function(quintile) {
            return(mean(quintile, na.rm = TRUE))
        })
        quintile_values <- as.data.frame(do.call('cbind', quintile_values))
        data_out <- cbind(factor_name, quintile_values)
        colnames(data_out) <- c('Factor', 1:n)
        return(data_out) # Actually quintile averages
    }

    # Use the monthly portfolio returns to calculate avg returns and their standard errors
    average_returns <- lapply(portfolio_returns, function(col) {
        model_data <- cbind(na.omit(col), 1)
        colnames(model_data) <- c('returns', 'one')
        model <- lm(formula = returns ~ one, data = model_data) # Explicit to avoid package error
        avg_ret <- model$coefficients[[1]]
        std_err <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6))[[1]]
        return(c(avg_ret, std_err))
    })

    # Calculate FF alpha and the corresponding NeweyWest t-statistic
    diff_returns <- portfolio_returns[,n + 1] # Returns of the diff portfolio
    diff_returns <- diff_returns[!is.na(diff_returns)] # Remove NAs
    diff_idx <- index(diff_returns)
    ff_data <- monthly_ff3_data[diff_idx, c('mkt_rf', 'smb', 'hml')]
    ff_lm_data <- cbind(diff_returns, ff_data)
    ff_model <- lm(diff_returns ~ mkt_rf + smb + hml, data = ff_lm_data) # FF3 regression
    ff_alpha <- ff_model$coefficients[[1]]
    ff_alpha_t <- suppressWarnings(ff_model$coefficients[[1]] / sqrt(NeweyWest(ff_model,
                    lag = 6))[[1]])
    ff <- c(ff_alpha, ff_alpha_t)

    # Output data frame construction
    metrics <- c('Average Returns', 'NeweyWest t-stat')
    average_returns <- as.data.frame(do.call('cbind', average_returns)) # To data frame
    data_out <- cbind(factor_name, metrics, average_returns, ff)
    colnames(data_out) <- c('Factor', 'Metric', portfolio_colnames, 'FF alpha')
    return(data_out)
}

# Specify the input data and return a prettified output as a data frame, in a fashion similar to the
#   table from lecture 10
#   input_data(list) - A list of the input data to be sorted by
#   factor_names(list) - A list of names of the factors to be used
#   Note: The rest of the arguments are similar as in the previous functions
getVariateSortOutput <- function(input_data, factor_names, monthly_returns_data,
        monthly_market_caps_data, monthly_ff3_data, n = 5, method = 'equal',
        get_quintile_values = FALSE) {
    input_length <- length(input_data)
    if (!input_length == length(factor_names)) {
        stop('The input data length must be the same as their names.')
    }
    data_out <- list()
    for (i in 1:input_length) { # Generate the portfolio sort for all factors
        input_sort <- input_data[[i]]
        input_factor <- factor_names[[i]]
        sort_results <- getUnivariateSort(input_sort, input_factor, monthly_returns_data,
            monthly_market_caps_data, monthly_ff3_data, n = n, method = method,
            get_quintile_values = get_quintile_values)
        data_out <- rbind(data_out, sort_results)
    }
    return(data_out)
}

In [10]:
# Univariate sort calculation

# Static variables
sort_output_input_data <- list(monthly_betas, monthly_sizes, monthly_volatilities)
sort_output_factor_names <- list('Beta', 'Size', 'Volatility')

# Quintile values
quintile_values <- getVariateSortOutput(sort_output_input_data, sort_output_factor_names,
    monthly_returns, monthly_market_caps, monthly_ff3, get_quintile_values = TRUE)

# Equally weighted univariate sort
equal_weighted_uni_sort <- getVariateSortOutput(sort_output_input_data, sort_output_factor_names,
    monthly_returns, monthly_market_caps, monthly_ff3, method = 'equal')

# Value weighted univariate sort
value_weighted_uni_sort <- getVariateSortOutput(sort_output_input_data, sort_output_factor_names,
    monthly_returns, monthly_market_caps, monthly_ff3, method = 'value')

In [11]:
print('Quintile average values:')
quintile_values

print('Equally weighted univariate sort:')
equal_weighted_uni_sort

print('Value weighted univariate sort:')
value_weighted_uni_sort

[1] "Quintile average values:"


Factor,1,2,3,4,5
Beta,0.4905917,0.7596982,0.9443506,1.149926,1.600434
Size,1.4118709,2.1240081,2.6941050,3.345308,4.575747
Volatility,4.3399710,5.5260423,6.4991150,7.838226,11.284661


[1] "Equally weighted univariate sort:"


Factor,Metric,1,2,3,4,5,5 - 1,FF alpha
Beta,Average Returns,0.01019706,0.01296348,0.01387961,0.01743733,0.01791254,0.007715479,0.00804043
Beta,NeweyWest t-stat,4.42983265,3.94260980,3.61452439,3.66966071,2.98504686,1.622798147,1.72160716
Size,Average Returns,0.02584814,0.01440903,0.01285798,0.01088162,0.00989257,-0.015955574,-0.01617407
Size,NeweyWest t-stat,4.80979881,3.28719327,2.74355440,3.17806714,2.78383631,-5.137289356,-5.32696900
Volatility,Average Returns,0.01039198,0.01260482,0.01367753,0.01630864,0.02046135,0.010069366,0.01124279
Volatility,NeweyWest t-stat,4.77743614,4.22110934,3.77197038,3.80774676,3.50895309,2.247916514,2.47839152


[1] "Value weighted univariate sort:"


Factor,Metric,1,2,3,4,5,5 - 1,FF alpha
Beta,Average Returns,0.011026175,0.01185709,0.01145078,0.01686160,0.02205218,0.01102600,0.01112093
Beta,NeweyWest t-stat,4.220911457,3.92946715,3.01254043,3.61487738,3.83270736,2.08846080,2.02934033
Size,Average Returns,0.031111269,0.01991171,0.01764821,0.01455486,0.01256208,-0.01854919,-0.01899525
Size,NeweyWest t-stat,5.160448981,4.45021609,3.84043153,4.31465765,3.93842395,-4.63052819,-5.15018358
Volatility,Average Returns,0.008998466,0.01106133,0.01507795,0.01589592,0.02491289,0.01591442,0.01647312
Volatility,NeweyWest t-stat,3.726076218,3.21370874,4.26418601,3.60268696,4.41874067,3.28093947,3.29045266


### Bivariate portfolio analysis

In [29]:
# Specify the characteristics and perform an independent bivariate portfolio sort
#   on two factors. Supports only value-weighted sort.
# Return a table with average returns with a corresponding Newey-West t-stat for all
#   quantiles, a difference portfolio, and lastly a FF-alpha with its t-stat.
# The arguments are similar to the univariate sort, only two factors are provided
#   instead of one. 'months_ahead' then specifies the #months to look-ahead for returns.
getBivariateSort <- function(first_sort_data, first_factor_name, second_sort_data,
                second_factor_name, monthly_returns_data, monthly_market_caps_data,
                monthly_ff3_data, n = 3, months_ahead = 1){
    # Static
    diff_portfolio <- paste0(n, ' - 1')
    bps <- c(0, 0.3, 0.7, 1)
    if (!length(bps) == (n + 1)) {
        stop('Incorrectly specified portfolio breakpoints')
    }

    # Match indexes
    index(first_sort_data) <- as.Date(as.yearmon(index(first_sort_data)), frac = 1)
    index(second_sort_data) <- as.Date(as.yearmon(index(second_sort_data)), frac = 1)
    factor_indexes_match <- index(first_sort_data) %in% index(second_sort_data)
    main_idx <- index(first_sort_data[factor_indexes_match])
    #Subset the sort data only to the observations where both values are available
    first_sort_data <- first_sort_data[main_idx]
    second_sort_data <- second_sort_data[main_idx]

    # Get portfolio returns for all portfolios
    portfolio_returns <- lapply(main_idx, function(current_month) {
        next_month <- as.Date(as.yearmon(current_month %m+% months(months_ahead)), frac = 1)
        if (!next_month %in% index(monthly_returns)){ 
            return(NA) # No monthly returns in this period
        }
        
        # Current month variables
        month_first_factor <- first_sort_data[current_month]
        month_second_factor <- second_sort_data[current_month]
        first_not_na <- !is.na(month_first_factor)
        second_not_na <- !is.na(month_second_factor)
        
        #Next month variables
        month_returns <- monthly_returns_data[next_month]
        month_market_caps <- monthly_market_caps_data[next_month]
        month_market_caps[is.na(month_market_caps)] <- 0
        
        # Skip in case of invalid data
        skip_conditions <- c(
            !any(first_not_na), # No data on first factor
            !any(second_not_na), # No data on second factor
            sum(month_market_caps) == 0 # No data on market capitalization
        )
        if (any(skip_conditions)) {
            # vec_length <- ifelse (get_quintile_values, n, n+1) # No diff in getting quintile values
            return(NA) # Change later depending on the structure of the outpu
        }

        # Find breakpoints
        first_breakpoints <- quantile(month_first_factor, bps, na.rm = TRUE)
        second_breakpoints <- quantile(month_second_factor, bps, na.rm = TRUE)

        # Create all combinations of the two breakpoints for the independent sort - nested list
        tuple_list <- apply(expand.grid(first_breakpoints, second_breakpoints), 1, list)
        current_month_out <- lapply(tuple_list, function(y) { # Iterate over all tuples
            first_bp_1 <- y[[1]][[1]] # 1st factor starting breakpoint
            second_bp_1 <- y[[1]][[2]] # 2nd factor staring breakpoint

            next_bp_idx_1 <- match(first_bp_1, first_breakpoints) + 1
            next_bp_idx_2 <- match(second_bp_1, second_breakpoints) + 1

            first_bp_2 <- first_breakpoints[next_bp_idx_1] # 1st factor ending breakpoint
            second_bp_2 <- second_breakpoints[next_bp_idx_2] # 2nd factor ending breakpoint

            if (is.na(first_bp_2) | is.na(second_bp_2)) { # Out of quantiles
                return('ooq')
            }

            #Construct the filters
            first_filter <- (month_first_factor < first_bp_2) &
                            (month_first_factor > first_bp_1) &
                            first_not_na
            second_filter <- (month_second_factor < second_bp_2) &
                             (month_second_factor > second_bp_1) &
                             second_not_na
            merged_filter <- (first_filter) & (second_filter)

            # Get returns for current month's weighted portfolio
            filtered_returns <- month_returns[, merged_filter]
            filtered_caps <- month_market_caps[,merged_filter]
            current_returns <- weighted.mean(t(filtered_returns), t(filtered_caps))
            return(current_returns)
        })
        current_month_out <- current_month_out[!current_month_out == 'ooq'] # Remove out of index quantiles
        current_month_out <- do.call('cbind', current_month_out)
        return(current_month_out)
    })
    portfolio_returns <- as.data.frame(do.call('rbind', portfolio_returns))
    rownames(portfolio_returns) <- main_idx
    tuple_grid <- expand.grid(c(1:n), c(1:n)) # Grid of all combinations of portfolio name tuples
    tuple_colnames <- sprintf('(%s,%s)', tuple_grid[,2], tuple_grid[,1]) # Coordinates ! (x,y)
    colnames(portfolio_returns) <- tuple_colnames 

    # Add returns of diff portfolios
    diff_cols <- c() # Diff portfolio column names

    for (i in 1:n) { # First factor diff portfolios
        temp_col <- paste0('first_diff_', i)
        diff_cols <- append(diff_cols, temp_col) # Store column name
        last_idx <- paste0('(', i, ',', n, ')') # Such as (1,3) - coordinates of matrix ! (x,y)
        first_idx <- paste0('(', i, ',' , 1, ')') # Such as (1,1)
        temp_returns <- portfolio_returns[,last_idx] - portfolio_returns[,first_idx]
        portfolio_returns[,temp_col] <- temp_returns # Assign column with diff portfolio returns
    }

    for (i in 1:n) { # Second factor diff portfolios
        temp_col <- paste0('second_diff_', i)
        diff_cols <- append(diff_cols, temp_col) # Store column name
        last_idx <- paste0('(', n, ',', i, ')') # Such as (3,1)
        first_idx <- paste0('(', 1, ',' , i, ')') 
        temp_returns <- portfolio_returns[,last_idx] - portfolio_returns[,first_idx]
        portfolio_returns[,temp_col] <- temp_returns # Assign column with diff portfolio returns
    }

    # Calculate average returns and newey-test stat for all portfolios (including diff)
    average_returns <- lapply(portfolio_returns, function(col) {
        model_data <- as.data.frame(cbind(na.omit(col), 1))
        colnames(model_data) <- c('returns', 'one')
        model <- lm(formula = returns ~ one, data = model_data) # Explicit to avoid package error
        avg_ret <- model$coefficients[[1]]
        std_err <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6))[[1]]
        return(c(avg_ret, std_err))
    })

    # Calculate FF3 alphas for both factors
    ff_alphas <- lapply(diff_cols, function(col){
        diff_returns <- portfolio_returns[,col] # Returns for this diff portfolio
        diff_returns <- diff_returns[!is.na(diff_returns)] # Remove NAs
        diff_idx <- index(diff_returns)
        ff_data <- monthly_ff3_data[diff_idx, c('mkt_rf', 'smb', 'hml')]
        ff_lm_data <- cbind(diff_returns, ff_data)
        ff_model <- lm(diff_returns ~ mkt_rf + smb + hml, data = ff_lm_data) # FF3 regression
        ff_alpha <- ff_model$coefficients[[1]]
        ff_alpha_t <- suppressWarnings(ff_model$coefficients[[1]] / sqrt(NeweyWest(ff_model,
                        lag = 6))[[1]])
        ff <- c(ff_alpha, ff_alpha_t)
        return(ff)
    })
    for (i in 1:(2*n)) { # Assign these to the average_returns object
        factor_colname <- paste0(diff_cols[i], '_ff_alpha')
        average_returns[factor_colname] <- ff_alphas[i]
    }
    
    # Averages and Newey-test t-stats of all portolios to a data frame
    all_returns <- lapply(average_returns, function(y){y[1]}) # Only returns
    all_nw <- lapply(average_returns, function(y){y[2]}) # Only N-W t-stats

    # Create and populate an output matrix/data frame
    temp_out <- matrix(nrow = n * 2 + 2 * 2, ncol = n + 2) # An empty matrix to store values
    fillTemp <- function(temp, rets, nw, dim, n) {
        it_rows <- n + 2 # Number of rows to iterate over
        for (i in 1:it_rows) { # Iterate over all rows
            in_quintile <- (i <= n)
            in_diff <- (n < i) & (i <= n + 1)
            if (in_quintile) { # Main quantiles
                ass_idx <- (i - 1) * 2 + 1 # Index of the row to assign to
                for (j in 1:n) { # Fill main quantiles
                    get_idx <- (i - 1) * 3 + j # Index of this metric in all_returns/all_nw
                    temp[ass_idx,j] <- rets[[get_idx]][1] # Return
                    temp[ass_idx + 1, j] <- nw[[get_idx]][1] # t-stat
                }
                temp[ass_idx, n + 1] <- rets[[n^2 + i]][1] # Difference portfolio returns (vert)
                temp[ass_idx + 1, n + 1] <- nw[[n^2 + i]][1] # Difference portfolio t-stat (vert)
                temp[ass_idx, n + 2] <- rets[[n^2 + 2 * n + i]][1] # FF alpha returns (vert)
                temp[ass_idx + 1, n + 2] <- nw[[n^2 + 2 * n + i]][1] # FF alpha t-stat (vert)
            } else if (in_diff){ # Diff portfolio
                ass_idx <- n * 2 + 1 # Index of the row to assign to
                for (j in 1:n) { # Across all quantiles
                    get_idx <- n^2 + n + j
                    temp[ass_idx, j] <- rets[[get_idx]][1] # Difference portfolio returns (hor)
                    temp[ass_idx + 1, j] <- nw[[get_idx]][1] # Difference portolio t-stat (hor)
                }
            } else { # FF alpha
                ass_idx <- n * 2 + 3
                for (j in 1:n) { # Across all quantiles
                    get_idx <- n^2 + 3 * n + j
                    temp[ass_idx, j] <- rets[[get_idx]][1] # FF alpha returns (hor)
                    temp[ass_idx + 1, j] <- nw[[get_idx]][1] # FF alpha t-stat (hor)
                }
            }
        }
        return(temp)
    }
    data_out <- fillTemp(temp_out, all_returns, all_nw, matrix_dim, n)
    # Output handling and transformation
    metrics <- rep(c('Average Returns', 'N-W t-stat'), n + 2)
    pf_names <- rep(c(paste(second_factor_name, 1:n), paste(second_factor_name, n, '- 1'),
                      paste(second_factor_name, n, '- 1 FF alpha')),
                    c(rep(2, n + 2))) # Repeat all elements up to n + 2
    data_out <- as.data.frame(data_out)
    data_out <- cbind(pf_names, metrics, data_out)
    data_cols <- c('Portfolio', 'Metrics',
                    paste(first_factor_name, 1:n), paste(first_factor_name, n, ' - 1'),
                    paste(first_factor_name, n, '- 1 FF alpha'))
    colnames(data_out) <- data_cols
    return(data_out)
    }

In [32]:
# Bivariate sort calculation
bivariate_sort_size_beta_1m <- getBivariateSort(monthly_sizes, 'Size', monthly_betas, 'Beta',
    monthly_returns, monthly_market_caps, monthly_ff3, months_ahead = 1)

bivariate_sort_size_vol_1m <- getBivariateSort(monthly_sizes, 'Size', monthly_volatilities, 'Volatility',
    monthly_returns, monthly_market_caps, monthly_ff3, months_ahead = 1)

bivariate_sort_size_beta_3m <- getBivariateSort(monthly_sizes, 'Size', monthly_betas, 'Beta',
    monthly_returns, monthly_market_caps, monthly_ff3, months_ahead = 3)

bivariate_sort_size_vol_3m <- getBivariateSort(monthly_sizes, 'Size', monthly_volatilities, 'Volatility',
    monthly_returns, monthly_market_caps, monthly_ff3, months_ahead = 3)

In [33]:
print('Value-weighted independent bivariate portfolio sort on Size-Beta - one-month-ahead:')
bivariate_sort_size_beta_1m

print('Value-weighted independent bivariate portfolio sort on Size-Volatility - one-month-ahead:')
bivariate_sort_size_vol_1m

print('Value-weighted independent bivariate portfolio sort on Size-Beta - one-quarter-ahead:')
bivariate_sort_size_beta_3m

print('Value-weighted independent bivariate portfolio sort on Size-Volatility - one-quarter-ahead:')
bivariate_sort_size_vol_3m

[1] "Value-weighted independent bivariate portfolio sort on Size-Beta - one-month-ahead:"


Portfolio,Metrics,Size 1,Size 2,Size 3,Size 3 - 1,Size 3 - 1 FF alpha
Beta 1,Average Returns,0.01919876,0.01153621,0.011168443,-0.008030316,-0.008179039
Beta 1,N-W t-stat,5.80399976,4.21963841,4.608185094,-4.142481701,-4.128645705
Beta 2,Average Returns,0.02346765,0.01512952,0.012496815,-0.010862846,-0.010934906
Beta 2,N-W t-stat,4.61350398,3.42535714,3.146754509,-4.283112763,-4.205341787
Beta 3,Average Returns,0.03678356,0.02184265,0.018020516,-0.018720312,-0.019019667
Beta 3,N-W t-stat,4.93210299,3.61515790,3.664616576,-4.441055305,-4.544623050
Beta 3 - 1,Average Returns,0.01758480,0.01030645,0.006618330,NA,NA
Beta 3 - 1,N-W t-stat,2.95180313,2.23383564,1.594264890,NA,NA
Beta 3 - 1 FF alpha,Average Returns,0.01781659,0.01039393,0.006787802,NA,NA
Beta 3 - 1 FF alpha,N-W t-stat,3.25001745,2.41093601,1.624272113,NA,NA


[1] "Value-weighted independent bivariate portfolio sort on Size-Volatility - one-month-ahead:"


Portfolio,Metrics,Size 1,Size 2,Size 3,Size 3 - 1,Size 3 - 1 FF alpha
Volatility 1,Average Returns,0.01643994,0.01147640,0.009625943,-0.006813993,-0.006893989
Volatility 1,N-W t-stat,5.85311071,4.38489412,3.545480809,-5.063665993,-5.329265345
Volatility 2,Average Returns,0.02246140,0.01356248,0.013301209,-0.009139589,-0.009034016
Volatility 2,N-W t-stat,4.84684124,3.19896385,3.691508415,-3.696426379,-3.472816103
Volatility 3,Average Returns,0.03539366,0.02191738,0.020683000,-0.014659645,-0.014902804
Volatility 3,N-W t-stat,4.74433834,3.81992853,4.255058085,-3.536627461,-3.464122866
Volatility 3 - 1,Average Returns,0.01895373,0.01044098,0.010692960,NA,NA
Volatility 3 - 1,N-W t-stat,3.15364804,2.44286165,2.850295618,NA,NA
Volatility 3 - 1 FF alpha,Average Returns,0.01939496,0.01052424,0.010922288,NA,NA
Volatility 3 - 1 FF alpha,N-W t-stat,3.58963323,2.60296099,2.987227487,NA,NA


[1] "Value-weighted independent bivariate portfolio sort on Size-Beta - one-quarter-ahead:"


Portfolio,Metrics,Size 1,Size 2,Size 3,Size 3 - 1,Size 3 - 1 FF alpha
Beta 1,Average Returns,0.01739184,0.01074536,0.010282906,-0.007108938,-0.006990089
Beta 1,N-W t-stat,5.43899858,4.05845069,3.961174375,-4.077719150,-3.791681159
Beta 2,Average Returns,0.02153292,0.01644569,0.013715712,-0.008583727,-0.008839728
Beta 2,N-W t-stat,4.42609111,3.60561578,3.604406972,-3.191658831,-3.286760029
Beta 3,Average Returns,0.03324822,0.02192668,0.016906635,-0.016115353,-0.016237918
Beta 3,N-W t-stat,5.00764637,3.86013458,3.462909606,-5.119046670,-5.172364396
Beta 3 - 1,Average Returns,0.01585637,0.01118133,0.006693591,NA,NA
Beta 3 - 1,N-W t-stat,3.22247989,2.62862943,1.724063486,NA,NA
Beta 3 - 1 FF alpha,Average Returns,0.01637266,0.01138736,0.007005083,NA,NA
Beta 3 - 1 FF alpha,N-W t-stat,3.31643358,2.67191591,1.800049130,NA,NA


[1] "Value-weighted independent bivariate portfolio sort on Size-Volatility - one-quarter-ahead:"


Portfolio,Metrics,Size 1,Size 2,Size 3,Size 3 - 1,Size 3 - 1 FF alpha
Volatility 1,Average Returns,0.01429021,0.01111133,0.010732016,-0.003558190,-0.003472729
Volatility 1,N-W t-stat,4.79214682,4.16759376,3.681590944,-1.725082392,-1.659077507
Volatility 2,Average Returns,0.02187472,0.01631375,0.013473190,-0.008917628,-0.009200941
Volatility 2,N-W t-stat,4.93129561,3.77876198,3.808747770,-4.188590896,-4.639024618
Volatility 3,Average Returns,0.03074627,0.02149312,0.019164125,-0.011342983,-0.011238041
Volatility 3,N-W t-stat,4.38530360,4.01087596,3.822552091,-3.076173267,-3.062969947
Volatility 3 - 1,Average Returns,0.01645606,0.01038179,0.009154044,NA,NA
Volatility 3 - 1,N-W t-stat,2.95016928,2.61168347,2.351491801,NA,NA
Volatility 3 - 1 FF alpha,Average Returns,0.01693171,0.01062653,0.009677824,NA,NA
Volatility 3 - 1 FF alpha,N-W t-stat,3.09087708,2.69259832,2.486779201,NA,NA


### Fama-MacBeth regression

In [ ]:
# Should control for all variables from the previous analysis
# Examine the effect of VOL independently
# Examine the effect of VOL using FF3 factors, momentum, and traded liquidity factor (download)
# All factors needed - returns, beta, size, volatility, FF3, momentum, illiquidity

In [39]:
# Define the explained data and the explanatory data and perform a Fama-Mac Beth regression
#   explained_data(xts) - Explained data in the regression - usually monthly returns
#   explanatory_data(list) - Explanatory data in the regression - a list of xts objects
#   explanatory_data_names(vector) - A vector with names of factors in the explanatory_data list
getFMBRegression <- function(explained_data, explanatory_data, explanatory_data_names) {
    if (!length(explanatory_data) == length(explanatory_data_names)){
        stop('The number of assigned factor names must match the length of the explanatory data')
    }
    # Static variables
    names(explanatory_data) <- explanatory_data_names
    output_names <- c('Intercept', explanatory_data_names)
    # Match indices

    main_idx <- as.Date(as.yearmon(index(explained_data)), frac = 1) # Load different data though
    # Get only those observations, where all data are present

    output_data <- NA
    return(output_data)
}

In [40]:
test_exp_data <- list(monthly_volatilities)
test_exp_data_names <- c('Volatility')
test_fmb_regression <- getFMBRegression(monthly_returns, test_exp_data, test_exp_data_names)